In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Papillary_Cell_Carcinoma"
cohort = "GSE85258"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma/GSE85258"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/GSE85258.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE85258.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/clinical_data/GSE85258.csv"
json_path = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
from typing import Optional, Callable, Dict, Any
import os
import json

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from Affymetrix U133 Plus2 microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Kidney_Papillary_Cell_Carcinoma)
trait_row = 2  # 'primary subtype' contains information about cancer type

# No age information available in the sample characteristics
age_row = None

# No gender information available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# For trait conversion - convert to binary (1 for Papillary RCC, 0 for other types)
def convert_trait(value):
    if value is None:
        return None
    # Extract value after colon if exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if it's Papillary RCC
    if 'Papillary RCC' in value:
        return 1
    elif 'ccRCC' in value:  # clear cell RCC
        return 0
    else:
        return None

# Age conversion function (not used but defined for completeness)
def convert_age(value):
    return None

# Gender conversion function (not used but defined for completeness)
def convert_gender(value):
    return None

# 3. Save Metadata - initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create the clinical data directly from the Sample Characteristics Dictionary
    # The dictionary structure has keys as row indices and values as lists of values
    clinical_data = pd.DataFrame({
        0: ['patient: 1', 'patient: 2', 'patient: 3', 'patient: 4', 'patient: 5', 'patient: 6', 
            'patient: 7', 'patient: 8', 'patient: 9', 'patient: 10', 'patient: 11', 'patient: 12', 
            'patient: 13', 'patient: 14', 'patient: 15'],
        1: ['tissue: Mets', 'tissue: Primary'],
        2: ['primary subtype: ccRCC', 'primary subtype: Papillary RCC']
    }).T  # Transpose to get features as rows, samples as columns
    
    # Select and process clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of processed clinical data:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the processed clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Saved clinical data to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I understand I need to develop code for this step that can work in conjunction with previous steps' code. This step requires analyzing a GEO dataset, extracting clinical features, and saving metadata.

```python
import pandas as pd
import numpy as np
import os
import json
import re
from typing import Optional, Callable, Dict, Any, List

# Let's first load the sample characteristics data from matrix file
# In GEO datasets, clinical data is often included in the matrix file
matrix_file = os.path.join(in_cohort_dir, "matrix.txt")
try:
    # Try to read the matrix file with different encodings
    for encoding in ['utf-8', 'latin1', 'ISO-8859-1']:
        try:
            with open(matrix_file, 'r', encoding=encoding) as f:
                lines = f.readlines()
            break
        except UnicodeDecodeError:
            continue
            
    # Extract sample characteristics
    sample_char_dict = {}
    sample_id_line = None
    in_sample_char = False
    
    for i, line in enumerate(lines):
        if line.startswith("!Sample_title"):
            sample_id_line = i
            
        if line.startswith("!Sample_characteristics_ch1"):
            in_sample_char = True
            if i not in sample_char_dict:
                sample_char_dict[i] = line.strip().split('\t')[1:]
        elif in_sample_char and line.startswith("!"):
            if not line.startswith("!Sample_characteristics_ch1"):
                in_sample_char = False
                
    # Load sample IDs
    if sample_id_line is not None:
        sample_ids = lines[sample_id_line].strip().split('\t')[1:]
    else:
        sample_ids = ["Sample_" + str(i) for i in range(len(next(iter(sample_char_dict.values()))))]
    
    # Create a clinical dataframe
    clinical_data = pd.DataFrame(index=sample_ids)
    for row, values in sample_char_dict.items():
        clinical_data.loc[:, f'characteristic_{row}'] = values
        
    # Print unique values for each characteristic to analyze
    print("Sample characteristic categories:")
    for col in clinical_data.columns:
        unique_values = clinical_data[col].unique()
        print(f"{col}: {unique_values[:5]}{'...' if len(unique_values) > 5 else ''}")
    
    # Based on the analysis of the data, determine availability and row indices
    is_gene_available = True  # Assuming this is a gene expression dataset (will be validated in a later step)
    
    # Inspect columns to find trait information (papillary RCC vs normal)
    found_trait = False
    trait_row = None
    for col in clinical_data.columns:
        unique_vals = [str(val).lower() for val in clinical_data[col].unique()]
        # Look for typical kidney cancer vs normal tissue indicators
        if any(['tumor' in str(val).lower() or 'normal' in str(val).lower() or 
                'papillary' in str(val).lower() or 'p-rcc' in str(val).lower() or
                'cancer' in str(val).lower() or 'carcinoma' in str(val).lower()
                for val in unique_vals]):
            trait_row = int(col.split('_')[-1])
            found_trait = True
            print(f"Found trait information in column {col}")
            break
    
    is_trait_available = found_trait
    
    # Inspect columns to find gender information
    found_gender = False
    gender_row = None
    for col in clinical_data.columns:
        unique_vals = [str(val).lower() for val in clinical_data[col].unique()]
        # Look for typical gender indicators
        if any(['male' in str(val).lower() or 'female' in str(val).lower() or
                'gender' in str(val).lower() or 'sex' in str(val).lower()
                for val in unique_vals]):
            gender_row = int(col.split('_')[-1])
            found_gender = True
            print(f"Found gender information in column {col}")
            break
    
    # Inspect columns to find age information
    found_age = False
    age_row = None
    for col in clinical_data.columns:
        unique_vals = clinical_data[col].unique()
        # Look for potential age data (numerical values with reasonable age range)
        age_patterns = [re.search(r'(\d+)\s*(?:y(?:ear)?s?)?', str(val)) for val in unique_vals]
        age_values = [int(match.group(1)) for match in age_patterns if match]
        
        if (age_values and 
            any(['age' in str(val).lower() for val in unique_vals]) or 
            (len(age_values) > 0 and min(age_values) >= 0 and max(age_values) <= 100)):
            age_row = int(col.split('_')[-1])
            found_age = True
            print(f"Found age information in column {col}")
            break
    
    # Define conversion functions for clinical features
    def convert_trait(value: str) -> int:
        """Convert trait value to binary (0 for normal, 1 for tumor)."""
        if value is None or pd.isna(value):
            return None
        value = str(value).split(':', 1)[-1].strip().lower() if ':' in str(value) else str(value).lower()
        if 'tumor' in value or 'pap' in value or 'p-rcc' in value or 'cancer' in value or 'carcinoma' in value:
            return 1
        elif 'normal' in value or 'non-tumor' in value or 'adjacent' in value or 'control' in value:
            return 0
        return None

    def convert_gender(value: str) -> int:
        """Convert gender value to binary (0 for female, 1 for male)."""
        if value is None or pd.isna(value):
            return None
        value = str(value).split(':', 1)[-1].strip().lower() if ':' in str(value) else str(value).lower()
        if 'female' in value or 'f' == value:
            return 0
        elif 'male' in value or 'm' == value:
            return 1
        return None

    def convert_age(value: str) -> float:
        """Convert age value to float."""
        if value is None or pd.isna(value):
            return None
        value = str(value).split(':', 1)[-1].strip() if ':' in str(value) else str(value)
        try:
            # Try to extract age as a number
            age_match = re.search(r'(\d+)', value)
            if age_match:
                return float(age_match.group(1))
            return None
        except:
            return None

    # Validate and save cohort information
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )

    # Extract clinical features if trait data is available
    if trait_row is not None:
        # Use the library function to extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age if age_row is not None else None,
            gender_row=gender_row,
            convert_gender=convert_gender if gender_row is not None else None
        )
        
        # Preview the selected clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Create output directory if it doesn't exist
        os.makedirs(os.path.dirname


### Step 4: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I've analyzed the current step requirements and the execution errors. Here's the corrected code to analyze the dataset and extract clinical features from GEO data.

```python
# Let's analyze the dataset and extract clinical features

# First, let's check what files are available in the cohort directory
print(f"Files in cohort directory: {os.listdir(in_cohort_dir)}")

# For GEO datasets, we typically need to analyze the series matrix file
matrix_files = [f for f in os.listdir(in_cohort_dir) if "_series_matrix.txt" in f]
if matrix_files:
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    print(f"Found series matrix file: {matrix_files[0]}")
    
    # Let's read the series matrix file to understand its structure
    with open(matrix_file, 'r') as f:
        # Read first 100 lines to understand file structure
        header_lines = [next(f) for _ in range(100) if f]
    
    for i, line in enumerate(header_lines):
        if line.startswith("!Sample_characteristics"):
            print(f"Line {i}: {line.strip()}")
else:
    print("No series matrix file found")

# Check if sample characteristics data is available
sample_char_path = os.path.join(in_cohort_dir, "sample_characteristics.json")
if os.path.exists(sample_char_path):
    with open(sample_char_path, 'r') as f:
        sample_characteristics = json.load(f)
    
    print("\nSample Characteristics Keys and Unique Values:")
    for key, values in sample_characteristics.items():
        print(f"Key {key}: {set(values)}")
else:
    print("No sample characteristics file found")
    sample_characteristics = {}

# Check background information
background_path = os.path.join(in_cohort_dir, "background.txt")
if os.path.exists(background_path):
    with open(background_path, 'r') as f:
        background_info = f.read()
    print("\nBackground Information:")
    print(background_info)
else:
    background_info = "No background information available"
    print(background_info)

# Check if gene expression data exists
is_gene_available = any("_series_matrix.txt" in f for f in os.listdir(in_cohort_dir))

# Load clinical data if it exists in a different format
pickle_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('.pickle')]
clinical_data = None

for pf in pickle_files:
    try:
        df = pd.read_pickle(os.path.join(in_cohort_dir, pf))
        if isinstance(df, pd.DataFrame):
            print(f"Found potential clinical data in {pf}")
            clinical_data = df
            break
    except:
        continue

if clinical_data is None:
    # If we still don't have clinical data, we'll need to extract it from the sample characteristics
    if sample_characteristics:
        # Convert the dictionary to a DataFrame for easier processing
        clinical_data = pd.DataFrame(sample_characteristics)
        print("Created clinical data from sample characteristics dictionary")
    else:
        print("No clinical data found")
        clinical_data = pd.DataFrame()  # Empty DataFrame as fallback

# Based on sample characteristics and background, determine availability of trait, age, and gender
trait_row = None
age_row = None
gender_row = None

# If we have the sample characteristics, search for trait, age, and gender
if sample_characteristics:
    for key, values in sample_characteristics.items():
        unique_values = list(set(values))
        value_str = ' '.join([str(v).lower() for v in unique_values])
        
        # For kidney papillary cell carcinoma, look for tumor/normal status
        if any(term in value_str for term in ["tumor", "cancer", "carcinoma", "normal", "histology", "diagnosis", "tissue", "type"]):
            print(f"Potential trait key {key}: {unique_values}")
            trait_row = key
        
        # Check for age information
        if any(term in value_str for term in ["age", "years", "yr"]):
            print(f"Potential age key {key}: {unique_values}")
            age_row = key
        
        # Check for gender/sex information
        if any(term in value_str for term in ["gender", "sex", "male", "female"]):
            print(f"Potential gender key {key}: {unique_values}")
            gender_row = key

# Define conversion functions for each variable
def convert_trait(value):
    """Convert trait values to binary (0 for normal, 1 for tumor/cancer)"""
    if value is None:
        return None
    
    # Extract the part after colon if present
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if not isinstance(value, str):
        value = str(value)
    
    value = value.lower()
    
    if any(term in value for term in ["tumor", "cancer", "carcinoma", "malignant", "papillary", "kirp"]):
        return 1
    elif any(term in value for term in ["normal", "healthy", "control", "adjacent", "non-tumor"]):
        return 0
    else:
        return None

def convert_age(value):
    """Convert age values to continuous numeric values"""
    if value is None:
        return None
    
    # Extract the part after colon if present
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Already a number
    if isinstance(value, (int, float)):
        return float(value)
    
    # Try to extract numeric age
    if isinstance(value, str):
        try:
            # Look for patterns like "XX years" or just numbers
            match = re.search(r'(\d+)(?:\s*years?|\s*y)?', value, re.IGNORECASE)
            if match:
                return float(match.group(1))
        except:
            pass
    
    return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    # Extract the part after colon if present
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if not isinstance(value, str):
        value = str(value)
    
    value = value.lower()
    
    if any(term in value for term in ["female", "f", "woman"]):
        return 0
    elif any(term in value for term in ["male", "m", "man"]):
        return 1
    else:
        return None

# Validate and save cohort info
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If trait data is available, extract clinical features
if trait_row is not None and clinical_data is not None and not clinical_data.empty:
    # Use the library function to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("\nPreview of selected clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname


### Step 5: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


### Step 6: Gene Identifier Review

In [ ]:
# Examining the gene identifiers from the gene expression data
# The identifiers shown (e.g., "1007_s_at", "1053_at", "117_at") appear to be 
# Affymetrix probe IDs from the U133 Plus 2.0 microarray platform
# These are not human gene symbols and need to be mapped to proper gene symbols

requires_gene_mapping = True


### Step 7: Gene Annotation

In [ ]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


### Step 8: Gene Identifier Mapping

In [ ]:
# 1. Identify the appropriate columns for mapping
# From the previous output, we can see:
# - In gene_data, we have probe IDs like "1007_s_at", "1053_at", etc. in the index
# - In gene_annotation, we have these columns:
#   - 'ID': Contains probe IDs matching our gene_data index 
#   - 'Gene Symbol': Contains the human gene symbols we need

# So we need to map from 'ID' to 'Gene Symbol'
prob_col = 'ID'
gene_col = 'Gene Symbol'

print("\nPreparing to map from Affymetrix probe IDs to gene symbols...")
print(f"Mapping from column '{prob_col}' to column '{gene_col}'")

# 2. Get gene mapping dataframe by extracting the two columns
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Created mapping dataframe with {len(mapping_df)} rows")

# Display sample of the mapping
print("\nSample of probe-to-gene mapping:")
print(preview_df(mapping_df, n=5))

# Count unique probes and genes in the mapping
unique_probes = mapping_df['ID'].nunique()
unique_genes = mapping_df['Gene'].str.split(' /// ').explode().nunique()
print(f"\nMapping contains {unique_probes} unique probes mapping to potentially {unique_genes} unique genes")

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data created with {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")

# Preview the gene data
print("\nPreview of gene expression data:")
print(preview_df(gene_data, n=5))

# Apply gene symbol normalization to handle synonyms
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 9: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
# This was already done and saved in the previous step, verified in the output
# We'll just make sure that gene_data is available for the rest of the processing

# 2. Extract clinical features since the previous extraction failed
print("\nExtracting clinical features...")
try:
    # Get the file paths for the SOFT file and matrix file again
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Obtain background information and clinical dataframe from matrix file
    background_info, clinical_data = get_background_and_clinical_data(
        matrix_file, 
        prefixes_a=['!Series_title', '!Series_summary', '!Series_overall_design'],
        prefixes_b=['!Sample_geo_accession', '!Sample_characteristics_ch1']
    )
    
    # For diagnostic purposes, check the unique values in each row
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    print("Sample Characteristics Dictionary:")
    print(sample_characteristics_dict)
    
    # Based on sample_characteristics_dict, define the row indices and conversion functions
    # Key 2 contains the primary subtype information (ccRCC vs Papillary RCC)
    trait_row = 2

    # Define conversion functions
    def convert_trait(value):
        if value is None:
            return None
        # Extract value after colon if exists
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Check if it's Papillary RCC
        if 'Papillary RCC' in value:
            return 1
        elif 'ccRCC' in value:  # clear cell RCC
            return 0
        else:
            return None

    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=None,  # No age data
        convert_age=None,
        gender_row=None,  # No gender data
        convert_gender=None
    )
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data extracted and saved to {out_clinical_data_file}")
    
    # For diagnostic purposes
    print("Clinical data preview:")
    print(preview_df(selected_clinical_df))
    
except Exception as e:
    print(f"Error extracting clinical features: {e}")
    # Create a basic empty clinical dataframe as a fallback
    selected_clinical_df = pd.DataFrame()

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    # Load the gene data if needed
    if 'gene_data' not in locals() or gene_data.empty:
        print("Loading gene data from saved file...")
        gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    
    # Check if clinical data is available and has the trait
    if not selected_clinical_df.empty:
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # 4. Handle missing values
        print("\nHandling missing values...")
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values, data shape: {linked_data.shape}")
        
        # 5. Determine whether the trait and demographic features are biased
        print("\nChecking for bias in features...")
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    else:
        print("Clinical data is not available. Cannot proceed with linking.")
        linked_data = pd.DataFrame()
        is_biased = True
except Exception as e:
    print(f"Error in linking data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 6. Final quality validation
print("\nPerforming final validation...")
note = ""
if not 'linked_data' in locals() or linked_data.empty:
    note = "Failed to link clinical and genetic data due to missing or incompatible data."
    is_biased = True
elif 'is_biased' in locals() and is_biased:
    note = "Dataset passed initial processing but contains severely biased trait distribution."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=not gene_data.empty if 'gene_data' in locals() else False,
    is_trait_available=not selected_clinical_df.empty if 'selected_clinical_df' in locals() else False,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
    note=note
)

# 7. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")